# Energy Mix
**Group 4**  
Jianyi Chen, Jincheng Hong, Tianyu Su, Yicheng Huang

## 1. Problem Statement

Planet 441 is a planet with natural conditions similar to those of B.C. It relies on a combination of renewable and non-renewable energy sources to meet its power and energy needs. While hydroelectricity is a significant contributor, other energy sources including coal and natural gas, as well as emerging renewable energy technologies such as wind and solar, also play a role. However, due to a special government type, the government has a fixed budget for the cost of energy production. Our goal is to develop a mathematical optimization model that minimizes pollution while ensuring that energy needs are met efficiently.

## 2. Variables and Parameters  

**Task:** Complete the table parameters and variables.

Hint: It may be helpful to know that GPa is gigapascals, which is a unit of pressure.  One megapascal is equal to one billion pascals and a Pascal in scientific units is: Pa $ = \frac{\textrm{kg}}{\textrm{m s}}.$


| Description | Symbol | Units | Type|
|-------------|--------|-------|-------|
| Concrete Compressive Strength | $Y$ | GPa | target (dependent) variable |
| Cement Component | $X1$ | kg/m$^3$| feature (independent) variable |
| Water Component | $X2$ | - | feature (independent) variable |
| Coarse Aggregate Component | $X3$ | kg/m$^3$| feature (independent) variable |
| Fine Aggregate Component | $X4$ | kg/m$^3$| feature (independent) variable |
| regression coefficent for intercept | $\beta_0$ | GPa | parameter |
| regression coefficent for Cement Component | $\beta_1$ | m$^2$/s | parameter |
| regression coefficent for Water Component | $\beta_2$ | kg/(m s) | parameter |
| regression coefficent for Coarse Aggregate Component | $\beta_3$ | m$^2$/s | parameter |
| regression coefficent for Fine Aggregate Component | $\beta_4$ | m$^2$/s | parameter |
| residural | $\epsilon$ | GPa | parameter |


## 3. Assumptions and Constraints

**Task:** State assumptions and constraints. 

- The model follows the linear model $Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \beta_3 X_3 + \beta_4 X_4 + \varepsilon$
- The average value of the error is 0: i.e., $\mathbb{E}(\epsilon_i) = 0$ for all $i$
- The variance of the error is constant: i.e., $Var(\epsilon_i) = \sigma^2$ for all $i$.
- The error $\vec{\epsilon}$ is a random sample from the normal distribution of zero mean and variance $\sigma^2$.
- The error is independent for each $\epsilon_i$ for all $i$.
- The features are independent of each other.
- The features are representative of the compressive strength.

## 4. Build Solutions

**Task:** Construct and run the linear regression model where `concrete_compressive_strength` is the output and `cement`,`water`, `course_aggregate`, and `fine_aggregate` are the inputs.

## 5. Simple Model

In [1]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum
import pandas as pd

# Load parameters from CSV
parameters_df = pd.read_csv("energy_parameters.csv", index_col=0)

# Define energy sources
energy_sources = parameters_df.index.tolist()

# Extract parameters from CSV
c = parameters_df["LCOE Cost ($/MWh)"].to_dict()
p = parameters_df["Pollution Cost (g/kWh CO2eq)"].to_dict()
f = parameters_df["Fixed Cost ($ per MW)"].to_dict()
l = parameters_df["Production Limit (MW)"].to_dict()
r = parameters_df["Renewable Indicator"].to_dict()
consumption_rate = parameters_df["Consumption Rate"].to_dict()

D = 45263.2  # Total energy demand (MW)
B = 9.36e9  # Total budget ($)
carbon_tax = 10  # Carbon tax ($ per tonne CO2eq)
R_min = 0.8  # Updated renewable energy target to 0.8

# Define variables
x = LpVariable.dicts("x", energy_sources, lowBound=0)  # Energy supply
y = LpVariable.dicts("y", energy_sources, lowBound=0)  # Energy extraction amount

# Create optimization model
model = LpProblem("Energy_Optimization", LpMinimize)

# Objective function: Minimize pollution cost including carbon tax
model += lpSum((p[i] / 1e6) * carbon_tax * x[i] for i in energy_sources)

# Constraint 1: Ensure energy demand is met (considering consumption rate)
model += lpSum(y[i] for i in energy_sources) >= D

# Constraint 2: Production limits
for i in energy_sources:
    model += x[i] <= l[i]
    model += y[i] == x[i] * consumption_rate[i]  # Extraction amount = Production * Consumption rate

# Constraint 3: Budget limitation
model += lpSum(f[i] * x[i] for i in energy_sources) <= B

# Constraint 4: Renewable energy ratio constraint (reformulated to avoid division)
renewable_energy = lpSum(y[i] * r[i] for i in energy_sources)
total_energy = lpSum(y[i] for i in energy_sources)
model += renewable_energy >= R_min * total_energy

# Solve the model
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/26ed181f95904ae88e723633b45eedd7-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/26ed181f95904ae88e723633b45eedd7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 18 COLUMNS
At line 54 RHS
At line 68 BOUNDS
At line 69 ENDATA
Problem MODEL has 13 rows, 10 columns and 30 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj 0 Primal inf 45554.345 (1)
10  Obj 3922.0567 Primal inf 2099115.5 (4)
Primal infeasible - objective value 3922.0567
PrimalInfeasible objective 3922.056729 - 10 iterations time 0.002

Result - Linear relaxation infeasible

Enumerated nodes:           0
Total iterations:           0


-1

## 6. Result for simple model

In [2]:
# Calculate total CO2 mass
total_co2_mass = sum(p[i] * x[i].varValue for i in energy_sources) / 1e6  # Convert from g to tonnes

# Store results
all_results = []
for i in energy_sources:
    all_results.append({
        "Energy Source": i,
        "Production (MW·h)": x[i].varValue,
        "Extraction (MW·h)": y[i].varValue,
        "Pollution Cost (g CO2eq)": p[i] * x[i].varValue,
        "Carbon Cost ($)": (p[i] / 1e6) * carbon_tax * x[i].varValue,
        "LCOE Cost ($)": c[i] * x[i].varValue,
        "Total CO2 Emissions (tonnes)": total_co2_mass
    })

# Save results to CSV
results_df = pd.DataFrame(all_results)
results_df.to_csv("renewable_optimization_results.csv", index=False)

# Print results
print(results_df)
print(f"Optimization completed for 80% renewable energy target. Total CO2 Emissions: {total_co2_mass:.2f} tonnes.")

  Energy Source  Production (MW·h)  Extraction (MW·h)  \
0         solar            905.264           814.7376   
1          wind            905.264           769.4744   
2         hydro        -273443.550       -259771.3700   
3          coal         432983.220        303088.2500   
4   natural_gas            452.632           362.1056   

   Pollution Cost (g CO2eq)  Carbon Cost ($)  LCOE Cost ($)  \
0              3.349477e+04         0.334948   7.830534e+04   
1              1.086317e+04         0.108632   9.641062e+04   
2             -4.101653e+07      -410.165325  -5.824348e+07   
3              4.329832e+08      4329.832200   3.204076e+07   
4              1.946318e+05         1.946318   2.444213e+04   

   Total CO2 Emissions (tonnes)  
0                    392.205677  
1                    392.205677  
2                    392.205677  
3                    392.205677  
4                    392.205677  
Optimization completed for 80% renewable energy target. Total CO2 Emission